In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
from matplotlib_venn import venn3

In [11]:
df_oa = pd.read_csv("/home/joe/Downloads/open_alex_query_results.csv")
df_s2 = pd.read_csv("../local/query/output/s2_title_scan_fix.tsv", sep='\t')

df_oa["doi"] = df_oa["doi"].astype('str').str.lower().str.replace('^https:\/\/doi.org\/','', regex=True)
df_s2["doi"] = df_s2["doi"].astype('str').str.lower().str.replace('^https:\/\/doi.org\/','', regex=True)

In [66]:
df_s2.shape

(870, 5)

In [13]:
print( df_s2["doi"].unique().size ) # Semantic Scholar Results Unique DOI
print( df_oa["doi"].unique().size ) # Open Alex Results Unique DOI
print( pd.concat([df_s2["doi"],df_oa["doi"]]).unique().size ) # Combined Results Unique DOI

614
1765
1803


In [14]:
df_oa_b = df_oa.iloc[:,[1,2,4,7]]

In [15]:
df_oa_b

,doi,title,publication_year,cited_by_count
0,10.14512/tatup.19.3.38,Carbon Capture and Storage – Politische und ge...,2010,2
1,10.4095/311151,Carbon capture and storage,2016,2
2,nan,Precambrian basement tectonics beneath a carbo...,2017,0
3,nan,Detecting Leakage from Carbon Capture and Stor...,2012,0
4,nan,Factors affecting stakeholder perception on Ca...,2011,0
...,...,...,...,...
2656,10.1016/j.jbiotec.2010.08.199,Assessment of a novel carbon capture and stora...,2010,0
2657,10.1088/1755-1307/6/17/172030,"RECCS : ecological, economic and structural co...",2009,12
2658,nan,Estimating the Possible Environmental Costs of...,2010,0
2659,10.1149/1.3429003,Possibility of SOFC using Biomass-fuel and Dis...,2010,2


In [16]:
df_s2["publication_year"] = df_s2["publication_year"].fillna(0).astype(int)

In [17]:
df_s2_b = df_s2.iloc[:,1:]

In [32]:
df_s2oa_b = df_s2_b.loc[df_s2_b["doi"] != "nan"].merge(df_oa_b.loc[df_oa_b["doi"] != "nan"], how="outer", on="doi", suffixes=["_s2","_openalex"])
df_s2oa_b["publication_year_match"] = df_s2oa_b["publication_year_openalex"] == df_s2oa_b["publication_year_s2"]
df_s2oa_b["citation_delta"] = df_s2oa_b["citation_count"] - df_s2oa_b["cited_by_count"]

In [35]:
df_s2oa_b.sort_values("citation_delta", ascending=False)

,doi,title_s2,publication_year_s2,citation_count,title_openalex,publication_year_openalex,cited_by_count,publication_year_match,citation_delta
268,10.1016/j.enpol.2007.03.009,Cost and performance of fossil fuel power plan...,2007.0,694.0,Cost and performance of fossil fuel power plan...,2007.0,611.0,True,83.0
339,10.1002/chin.201248258,Atmospheric Chemistry and Environmental Impact...,2012.0,62.0,Atmospheric Chemistry and Environmental Impact...,2012.0,2.0,True,60.0
550,10.1007/s10584-005-3484-7,Carbon Capture and Storage From Fossil Fuels a...,2004.0,362.0,Carbon capture and storage from fossil fuels a...,2006.0,304.0,False,58.0
594,10.1016/j.pecs.2011.11.002,Carbon capture and storage using alkaline indu...,2012.0,391.0,Carbon capture and storage using alkaline indu...,2012.0,333.0,True,58.0
286,10.1007/s10584-010-9832-7,The feasibility of low CO2 concentration targe...,2010.0,288.0,The feasibility of low CO2 concentration targe...,2010.0,234.0,True,54.0
...,...,...,...,...,...,...,...,...,...
1805,10.1260/0958-305x.23.2-3.409,NaN,NaN,NaN,Carbon Capture and Storage - A Pragmatic Way t...,2012.0,0.0,False,NaN
1806,10.25560/14512,NaN,NaN,NaN,Carbon Capture and Storage the Clean Developme...,2012.0,0.0,False,NaN
1807,10.1016/j.jbiotec.2010.08.199,NaN,NaN,NaN,Assessment of a novel carbon capture and stora...,2010.0,0.0,False,NaN
1808,10.1088/1755-1307/6/17/172030,NaN,NaN,NaN,"RECCS : ecological, economic and structural co...",2009.0,12.0,False,NaN


In [49]:
np.where(np.in1d(df_s2oa_b["publication_year_openalex"],[2012,2015]),True,False)

array([ True, False, False, ..., False, False, False])

In [52]:
np.where((df_s2oa_b["publication_year_openalex"] >= 2012) & (df_s2oa_b["publication_year_openalex"] <= 2015),True,False)

array([ True, False, False, ..., False, False, False])

In [56]:
np.where(df_s2oa_b["publication_year_openalex"].isin([2012,2015]),True,False)

array([ True, False, False, ..., False, False, False])

In [63]:
np.where(df_s2oa_b["publication_year_openalex"].between(2012,2015),True,False)

array([ True, False, False, ..., False, False, False])